In [4]:
# Prep ... import libraries
import pandas as pd
import requests
from pathlib import Path

# Import API key & set base URL
from keys import api_key
url="https://api.walkscore.com/score?format=json&address=skip"

# Define variables
FindState = "NY"

In [3]:
# Import CSV
data_df = pd.read_csv("resources/obesity_data_ALL_states.csv")
data_df = pd.DataFrame(data_df)

# Insert new column at end to import Walk Score
data_df["Walk Score"]=''

# Clean up dataframe by removing any rows with no data
data_df_cleaned=data_df.dropna()

data_df_cleaned.head()

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,TX,Houston,4835000-48201451401,4835000,48201451401,25.5,3330,29.750367,-95.612367,
1,NY,New York,3651000-36085017010,3651000,36085017010,26.3,8583,40.551099,-74.196047,
2,NY,Syracuse,3673000-36067005601,3673000,36067005601,23.1,1473,43.024362,-76.121220,
3,UT,Sandy,4967440-49035112613,4967440,49035112613,23.7,4164,40.570109,-111.844681,
4,CA,Los Angeles,0644000-06037190902,644000,6037190902,30.2,3960,34.094395,-118.319013,


In [3]:
# Let's do API calls for just ONE state
# We define FindState in first block...
one_state = data_df_cleaned.loc[data_df_cleaned["StateAbbr"] == FindState, :]

# reset index because of dropped rows
one_state.reset_index(drop=True, inplace=True)

one_state.head(10)

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,NY,New York,3651000-36085017010,3651000,36085017010,26.3,8583,40.551099,-74.196047,
1,NY,Syracuse,3673000-36067005601,3673000,36067005601,23.1,1473,43.024362,-76.121220,
2,NY,New York,3651000-36005021301,3651000,36005021301,39.7,1201,40.845508,-73.919779,
3,NY,New York,3651000-36005001900,3651000,36005001900,37.4,1917,40.801243,-73.909588,
4,NY,New York,3651000-36047041600,3651000,36047041600,23.5,2324,40.602411,-73.963486,
5,NY,New York,3651000-36047006800,3651000,36047006800,24.5,4870,40.635175,-74.026230,
6,NY,New York,3651000-36047005800,3651000,36047005800,22.2,3152,40.619089,-74.032188,
7,NY,New York,3651000-36061012000,3651000,36061012000,15.9,3971,40.766926,-73.964767,
8,NY,New York,3651000-36005022404,3651000,36005022404,30.8,4509,40.852745,-73.866301,
9,NY,New York,3651000-36061027100,3651000,36061027100,25.0,8196,40.851939,-73.934290,


In [ ]:
# Now let's iterate thru that ONE state and get the data

for index,row in one_state.iterrows():

    try:
        lat = str(row["Lat"])
        lon = str(row["Lon"])
        query_url = url+"&lat="+lat+"&lon="+lon+"&wsapikey="+api_key
        walk_response = requests.get(query_url)
        walk_json = walk_response.json() 
        if walk_json["status"] == 41:
            print("Daily API quota exceeded")
            print("^^^^^^^^^^^^^^^^^^^^^^^^")
            break
            
        print("Processing LAT %s, LON, %s" % (lat, lon))
        one_state.loc[index, "Walk Score"] = walk_json["walkscore"]
    
    except:
        print("Data not found. Skipping")
        pass

one_state.head()



Daily API quota exceeded
^^^^^^^^^^^^^^^^^^^^^^^^


,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,NY,New York,3651000-36085017010,3651000,36085017010,26.3,8583,40.551099,-74.196047,
1,NY,Syracuse,3673000-36067005601,3673000,36067005601,23.1,1473,43.024362,-76.121220,
2,NY,New York,3651000-36005021301,3651000,36005021301,39.7,1201,40.845508,-73.919779,
3,NY,New York,3651000-36005001900,3651000,36005001900,37.4,1917,40.801243,-73.909588,
4,NY,New York,3651000-36047041600,3651000,36047041600,23.5,2324,40.602411,-73.963486,


In [15]:
# Save output to CSV
output_csv = f'resources/{FindState}_complete.csv'
one_state.to_csv(output_csv, index=False)